In [58]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [59]:
(Xtrain, ytrain), (Xtest,ytest) = mnist.load_data()

In [60]:
# reshape and normalize
Xtrain = Xtrain.reshape(-1,28,28,1).astype("float32") / 255.0
Xtest = Xtest.reshape(-1,28,28,1).astype("float32") / 255.0


Model Subclassing



In [61]:
class CNNBlock(layers.Layer): #keep track of everything
  def __init__(self,out_channels,kernel_size=3):
    super(CNNBlock,self).__init__()
    self.conv = layers.Conv2D(out_channels,kernel_size,padding='same')
    self.bn = layers.BatchNormalization()
  
  #creating cnn -> batchnorm -> relu block
  def call(self, input_tensor, training=False):
    #running the input_tensor through the layers
    x = self.conv(input_tensor)
    x = self.bn(x, training=training)
    x = tf.nn.relu(x)
    return x


model = keras.Sequential([
              CNNBlock(32),
              CNNBlock(64),
              CNNBlock(128),
              layers.Flatten(),
              layers.Dense(10)  #output layer
])



In [62]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ["accuracy"]
)

In [63]:
model.fit(Xtrain,ytrain,batch_size=64,epochs=1)

938/938 [==============================] - 730s 778ms/step - loss: 0.5410 - accuracy: 0.9466


In [64]:
model.evaluate(Xtest,ytest,batch_size=64)

157/157 [==============================] - 31s 195ms/step - loss: 0.2028 - accuracy: 0.9680


[0.2028096616268158, 0.9679999947547913]

Creating Resnet like model structure. 

In [65]:
class Resblock(layers.Layer):
  def __init__(self,channels):
    super(Resblock,self).__init__()
    self.cnn1 = CNNBlock(channels[0],3)
    self.cnn2 = CNNBlock(channels[1],3)
    self.cnn3 = CNNBlock(channels[2],3)
    self.pooling = layers.MaxPooling2D()
    self.identity_mapping = layers.Conv2D(channels[1],1,padding='same')

  def call(self,input_tensor,training=False):
    x = self.cnn1(input_tensor,training=training)
    x = self.cnn2(x,training=training)
    x = self.cnn3(
        x + self.identity_mapping(input_tensor),training=training
        )
    return self.pooling(x)

In [66]:
class Resnet_Like(keras.Model):
  def __init__(self,num_classes=10):
    super(Resnet_Like,self).__init__()
    self.block1 = Resblock([32,32,64])
    self.block2 = Resblock([128,128,256])
    self.block3 = Resblock([128,256,512])
    self.pool = layers.GlobalAveragePooling2D()
    self.classifier = layers.Dense(num_classes)
  
  def call(self,input_tensor,training=False):
    x = self.block1(input_tensor, training=training)
    x = self.block2(x,training=training)
    x = self.block3(x,training=training)
    x = self.pool(x)
    return self.classifier(x)

  

In [67]:
model = Resnet_Like(num_classes=10)

In [68]:
model.compile(keras.optimizers.Adam(),
              loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [63]:
model.fit(Xtrain,ytrain,batch_size=64,epochs=1)

938/938 [==============================] - 2072s 2s/step - loss: 0.0839 - accuracy: 0.9750


In [7]:
# reshape and normalize
Xtrain = Xtrain.reshape(-1,28*28).astype("float32") / 255.0
Xtest = Xtest.reshape(-1,28*28).astype("float32") / 255.0

  Custom Layers

In [4]:
class MyModel(keras.Model):
  def __init__(self,num_classes=10):
    super(MyModel,self).__init__()
    self.dense1 = layers.Dense(64)
    self.dense2 = layers.Dense(num_classes)
  
  def call(self,input_tensor):
    x = tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)

  

In [8]:
model = MyModel()
model.compile(optimizer=keras.optimizers.Adam(),
              loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy'])


In [11]:
model.fit(Xtrain,ytrain,batch_size=64,epochs=3)

Epoch 1/3
938/938 [==============================] - 2s 2ms/step - loss: 0.5989 - accuracy: 0.8536
Epoch 2/3
938/938 [==============================] - 2s 2ms/step - loss: 0.4691 - accuracy: 0.8787
Epoch 3/3
938/938 [==============================] - 2s 2ms/step - loss: 0.4064 - accuracy: 0.8908


In [12]:
model.evaluate(Xtest,ytest,batch_size=64)

157/157 [==============================] - 0s 2ms/step - loss: 0.3722 - accuracy: 0.8997


[0.3721504807472229, 0.8996999859809875]

Creating Custom Dense Layer

In [21]:
class Dense(layers.Layer):
  def __init__(self,units,input_dim):
    super(Dense,self).__init__()
    self.w = self.add_weight(
        name='w',
        shape=(input_dim,units),
        initializer='random_normal',
        trainable=True,
    )

    self.b = self.add_weight(
        name='b', 
        shape=(units,),
        initializer='zeros',
        trainable=True
    )
  
  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b


In [27]:
class MyModel1(keras.Model):
  def __init__(self,num_classes=10):
    super(MyModel1,self).__init__()
    self.dense1 = Dense(64,784)
    self.dense2 = Dense(10,64)
  
  def call(self,input_tensor):
    x = tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)

In [30]:
model1 = MyModel1()

In [32]:
model1.compile(optimizer=keras.optimizers.Adam(),
               loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

In [33]:
model1.fit(Xtrain,ytrain,batch_size=32,epochs=3)

Epoch 1/3
1875/1875 [==============================] - 4s 2ms/step - loss: 1.4638 - accuracy: 0.5976
Epoch 2/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.6978 - accuracy: 0.8166
Epoch 3/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5069 - accuracy: 0.8640


In [34]:
model1.evaluate(Xtest,ytest,batch_size=32)

313/313 [==============================] - 1s 2ms/step - loss: 0.4436 - accuracy: 0.8786


[0.4435982406139374, 0.878600001335144]

Updating our model such that it works regardless of the input dimension

In [53]:
class Dense(layers.Layer):
  def __init__(self,units):
    super(Dense,self).__init__()
    self.units = units
  
  def build(self,input_shape):
    self.w = self.add_weight(
        name='w',
        shape=(input_shape[-1],self.units),
        initializer='random_normal',
        trainable=True,
    )

    self.b = self.add_weight(
        name='b', 
        shape=(self.units),
        initializer='zeros',
        trainable=True
    )

  def call(self,inputs):
    return tf.matmul(inputs, self.w) + self.b

In [49]:
class MyModel2(keras.Model):
  def __init__(self,num_classes=10):
    super(MyModel2,self).__init__()
    self.dense1 = Dense(64)
    self.dense2 = Dense(10)
  
  def call(self,input_tensor):
    x = tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)

In [54]:
model2 = MyModel2()

In [55]:
model2.compile(optimizer=keras.optimizers.Adam(),
               loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

In [56]:
model2.fit(Xtrain,ytrain,batch_size=32,epochs=3)

Epoch 1/3
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3236 - accuracy: 0.6550
Epoch 2/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5843 - accuracy: 0.8467
Epoch 3/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4377 - accuracy: 0.8811


In [57]:
model2.evaluate(Xtest,ytest,batch_size=32)

313/313 [==============================] - 1s 1ms/step - loss: 0.3873 - accuracy: 0.8950


[0.38734304904937744, 0.8949999809265137]